In [1]:
import tensorflow as tf


In [2]:
from tensorflow.keras import layers, models


In [3]:
from pythainlp.tokenize import word_tokenize


In [4]:
from pythainlp.corpus import thai_stopwords


In [5]:
from pythainlp.util import normalize


In [6]:
import numpy as np


In [29]:
# ข้อมูลตัวอย่าง (เปลี่ยนเป็นข้อมูลข่าวภาษาไทยที่มีคลาสบวกและลบ)
texts = ["ข่าวดี: การวิจัยล่าสุดพบว่าการออกกำลังกายเป็นสิ่งที่ดีต่อสุขภาพ",
         "ข่าวไม่ดี: การศึกษาล่าสุดพบว่าการนั่งนานๆ อาจทำให้เสี่ยงต่อการเป็นโรคหลอดเลือดในสมอง",
         "ข่าวไม่ดี: เสียชีวิตอีกแล้ว พันกว่าราย",
         "ข่าวไม่ดี: เจ็บป่วยยังไม่หาย"]

In [30]:

# สร้างตัวแปรเก็บคำหยุด
stopwords = list(thai_stopwords())

In [31]:
# สร้างลิสต์เก็บข้อความที่ถูกตัดคำแล้ว
tokenized_texts = []

In [32]:
# ตัดคำและทำความสะอาดข้อมูล
for text in texts:
    tokens = word_tokenize(normalize(text))
    tokens = [token for token in tokens if token not in stopwords]
    tokenized_texts.append(tokens)

In [33]:
# สร้าง dictionary ของคำศัพท์
word_index = {}
index = 1
for tokens in tokenized_texts:
    for token in tokens:
        if token not in word_index:
            word_index[token] = index
            index += 1

In [34]:
# แปลงข้อความเป็นลิสต์ของตัวเลขโดยใช้ dictionary ของคำศัพท์
sequences = []
for tokens in tokenized_texts:
    sequence = [word_index[token] for token in tokens]
    sequences.append(sequence)


In [35]:
# สร้างข้อมูลเข้าให้เป็นขนาดเท่ากัน
max_len = max(len(sequence) for sequence in sequences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)


In [37]:
# สร้างโมเดล
model = models.Sequential([
    layers.Embedding(input_dim=len(word_index)+1, output_dim=16, input_length=max_len),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [38]:

# คอมไพล์โมเดล
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [40]:
# ฝึกโมเดล
labels = np.array([1, 0])  # เปลี่ยนตามคลาสของข้อมูลข่าว
model.fit(padded_sequences, labels, epochs=10, batch_size=32)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 4
'y' sizes: 2


In [28]:
# ทดสอบโมเดล
test_texts = ["ข่าวไม่ดี: เสียชีวิตไปเกือบร้อย"]
test_sequences = []
for test_text in test_texts:
    tokens = word_tokenize(normalize(test_text))
    tokens = [token for token in tokens if token not in stopwords]
    sequence = [word_index[token] for token in tokens if token in word_index]
    test_sequences.append(sequence)

# ตรวจสอบว่ามี tokens ใน test_text หรือไม่
if len(test_sequences) > 0:
    test_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len)
    predictions = model.predict(test_padded_sequences)
    print("ซึ่งค่านี้อยู่ในช่วงระหว่าง 0 ถึง 1 โดยค่ามากกว่า 0.5 หมายถึงโมเดลคาดเดาว่าข่าวนั้นเป็นข่าวลบ (หรือคลาส 0) ในขณะที่ค่าน้อยกว่า 0.5 หมายถึงโมเดลคาดเดาว่าข่าวนั้นเป็นข่าวบวก (หรือคลาส 1)")
    print("Predictions:", predictions)
else:
    print("No valid tokens found in the test text.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
ซึ่งค่านี้อยู่ในช่วงระหว่าง 0 ถึง 1 โดยค่ามากกว่า 0.5 หมายถึงโมเดลคาดเดาว่าข่าวนั้นเป็นข่าวลบ (หรือคลาส 0) ในขณะที่ค่าน้อยกว่า 0.5 หมายถึงโมเดลคาดเดาว่าข่าวนั้นเป็นข่าวบวก (หรือคลาส 1)
Predictions: [[0.50901484]]
